<a href="https://colab.research.google.com/github/castudil/Machine-Learning/blob/main/S04-KNN/KNN_Diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KNN - Predict whether a person will have diabetes or not

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [14]:
dataset = pd.read_csv('diabetes.csv')

In [15]:
len(dataset) ## esta función simplemente no se entrega la cantidad de instancias que existen en el conjunto de datos

768

In [16]:
dataset.head(n=8) ## Head se pide prestada de Unix y muestra por defecto las primeras N líneas del conjunto de datos

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0


En esta etapa tenemos que hacernos cargo de los datos porque nos dimos cuenta que tienen algunas falencias. Por ejemplo existen ceros en lugares donde existían los datos faltantes. El problema es que hay algunas columnas que el cero indica algo real, por ejemplo el número de embarazos previos. Para poder resolver este problema debemos identificar cuáles son las columnas que tienen ceros y que deberían ser datos faltantes. Luego de identificarlos copiamos los datos faltantes en esas celdas y luego generaremos una estrategia de imputación de datos, para reparar los valores de esas celdas. La idea va hacer tratar de estimar un valor adecuado para esas celdas.

Una forma muy sencilla de realizar esto consiste en calcular las medias por columnas.

In [17]:
# Replace zeroes
zero_not_accepted = ['Glucose', 'BloodPressure', 'SkinThickness', 'BMI', 'Insulin']

In [18]:
for column in zero_not_accepted:
    dataset[column] = dataset[column].replace(0, np.NaN) ## en las columnas especificadas vamos a ir a buscar ceros y vamos a copiar el codigo para indicar un valor faltante
    mean = int(dataset[column].mean(skipna=True)) ## vamos a calcular las medias por columna.
    dataset[column] = dataset[column].replace(np.NaN, mean) ## vamos a imputar los valores faltantes, utilizando las medidas que calculamos recién

In [19]:
dataset.head(n=8)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,155.0,33.6,0.627,50,1
1,1,85.0,66.0,29.0,155.0,26.6,0.351,31,0
2,8,183.0,64.0,29.0,155.0,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
5,5,116.0,74.0,29.0,155.0,25.6,0.201,30,0
6,3,78.0,50.0,32.0,88.0,31.0,0.248,26,1
7,10,115.0,72.0,29.0,155.0,35.3,0.134,29,0


En esta etapa ya estamos contentos con la calidad de los datos ya están preparados y listos para hacer modelados. 

Entonces, nos corresponde diferenciar entre los datos que van a ser utilizados para el entrenamiento y aquellos que se van a usar para la validación.

train_test_split se encarga automáticamente de dividir los conjuntos de entrenamiento y validación, lo único que hacemos es especificar cuáles son las proporciones que queremos


In [20]:
# split dataset
X = dataset.iloc[:, 0:8] ## X corresponde a las variables independientes
y = dataset.iloc[:, 8] ## y se usa para denotar la variable objetivo o Target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)


In [21]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))


614
614
154
154


In [24]:
614+154

768

In [23]:
154/(614+154)

0.20052083333333334

Para asegurarnos podemos hacer un Hit de los datos que ya hemos obtenido a través de la division. Podemos observar que la primera columna de alguna manera conserva la instancia o el identificador de la instancia inicial en el conjunto diabetes. Esta información puede ser valiosa al momento de hacer trazabilidad. Observen también que en particular, en la columna insulina, obtenemos algunos valores que fueron imputados  

In [25]:
X_test.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
661,1,199.0,76.0,43.0,155.0,42.9,1.394,22
122,2,107.0,74.0,30.0,100.0,33.6,0.404,23
113,4,76.0,62.0,29.0,155.0,34.0,0.391,25
14,5,166.0,72.0,19.0,175.0,25.8,0.587,51
529,0,111.0,65.0,29.0,155.0,24.6,0.660,31


La estandarización de los datos o normalización tiene que ver con dejar todos los valores en un rango comparable. En general es un paso importante que se aplica casi todas las veces al momento de hacer modela mientoEn general es un paso importante que se aplica casi todas las veces al momento de hacer modela miento. Razón es que si se fijan algunas de las columnas tienen magnitudes de valores mayores, por ejemplo glucosa. Si nosotros calculamos distancias utilizando los vectores originales aquellas columnas con magnitudes mayores tendrán una mayor implicancia en el resultado. En otras palabras aquellas columnas con rangos mucho menores, por ejemplo la penúltima columna sería insignificante con respecto a los cambios que se producen en aquellas columnas con rangos más amplios. 

In [26]:
#Feature scaling
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [29]:
X_train

array([[ 0.90832902,  0.93641795,  0.44764174, ...,  0.36863635,
         0.67740401,  1.69955804],
       [ 0.03644676, -0.81630913, -1.05200558, ..., -0.63294341,
        -0.07049698, -0.96569189],
       [-1.12606292,  1.43247278,  1.44740662, ...,  2.81535261,
        -0.11855487, -0.88240283],
       ...,
       [ 0.03644676, -0.91552009, -0.63543688, ..., -1.13373329,
        -0.95656442, -1.04898095],
       [ 2.0708387 , -1.21315299,  0.11438678, ..., -0.36108605,
        -0.50001442,  0.11706589],
       [ 0.32707418,  0.47343344,  0.7808967 , ..., -0.08922869,
         0.52121586,  2.94889395]])

In [30]:
# Define the model: Init K-NN
classifier = KNeighborsClassifier(n_neighbors=11, p=2,metric='euclidean')

In [31]:
# Fit Model
classifier.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=11)

Para realizar predicciones entonces vamos a utilizar nuestro conjunto de validación. Dado que el conjunto de validación tiene 154 instancias, el modelo contesta 154 etiquetas en ceros y unos que es los valores posibles que pueden tener

In [32]:
# Predict the test set results
y_pred = classifier.predict(X_test)
y_pred

array([1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Es difícil determinar qué tan efectivo fue el modelo al revisar solamente ceros y unos. Entonces por ello necesitamos desarrollar una estrategia de validación, que nos entregue algún resumen estadístico que podamos comprender.

Una técnica muy común es la llamada matriz de confusión. Esta matriz tiene tantas filas como etiquetas, y las columnas también tienen etiquetas pero en las filas tienen las columnas reales los valores reales de las instancias mientras que las columnas se tienen las que arrojó el sistema. Cuando estas etiquetas coinciden se asume que hubo un acierto. Análoga mente cuando estas etiquetas difieren se considera que hubo un error

In [33]:
# Evaluate Model
cm = confusion_matrix(y_test, y_pred)
print (cm)
print(f1_score(y_test, y_pred))

[[94 13]
 [15 32]]
0.6956521739130436


En concreto vemos una matriz de 2 × 2, porque el problema tenía dos etiquetas. Por ejemplo aquí nos dicen que 94 instancias que eran originalmente no diabetes fueron identificadas como tal, sin embargo 13 de ellas fueron identificadas como diabetes, es decir se cometió un error.

General al sumar los elementos de las diagonales tendremos una estimación de cuántas veces tuvimos éxito. De allí se puede sacar uno de los estadísticos más importantes para comprender la efectividad del modelo predictivo. A esa estadístico se le conoce como certeza.

In [ ]:
print(accuracy_score(y_test, y_pred))

0.8181818181818182


En este caso la certeza de 0,81 nos dice que el modelo aceptó en un 81% de las veces. Pero podemos hacer algunos cálculos matemáticos para convencernos de ello.

In [34]:
(94+32)+(15+13)

154

In [35]:
(94+32)/154

0.8181818181818182